In [1]:
import h5py
import numpy as np
import scipy.io
import cv2
import glob
import os

In [3]:
def loadformat1(filepath, count):
    filenames = glob.glob(filepath)
#     filenames = filenames[:110000]

    for i, filename in enumerate(filenames):
        img = cv2.imread(filename)
        if len(img) >= 16 and len(img[0]) >= 48:
            cropped = img[0:16, 0:16, :]
            res = res = cv2.resize(cropped, dsize=(32, 32), interpolation=cv2.INTER_CUBIC)
            cv2.imwrite('nondigits/{}.png'.format(count), res)
            count += 1
            if count % 10000 == 0:
                print(count)
    return count

count = 0
count = loadformat1('svhn_format1/train/*.png', count)
print("TRAIN --- ", count)
count = loadformat1('svhn_format1/test/*.png', count)
print("TEST --- ", count)
count = loadformat1('svhn_format1/extra/*.png', count)
print("EXTRA --- ", count)

10000
20000
30000
TRAIN ---  31844
40000
TEST ---  44823
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
EXTRA ---  236192


In [4]:
img = cv2.imread('nondigits/1.png')
print(img.shape)

(32, 32, 3)


In [4]:
def createCombinedFormat2Matfile():
    filename = 'svhn_format2/train_32x32.mat'
    data = scipy.io.loadmat(filename)
    Xtrain = data['X']
    Xtrain = np.rollaxis(Xtrain, 3)
    ytrain = data['y'].flatten()
    ytrain[ytrain == 10] = 0
    ytrain = ytrain.reshape((len(ytrain), 1))
    print(Xtrain.shape)
    print(ytrain.shape)
    filename = 'svhn_format2/test_32x32.mat'
    data = scipy.io.loadmat(filename)
    Xtest = data['X']
    Xtest = np.rollaxis(Xtest, 3)
    ytest = data['y']
    ytest[ytest == 10] = 0
    ytest = ytest.reshape((len(ytest), 1))
    print(Xtest.shape)
    print(ytest.shape)
    
    Xfinal = np.concatenate((Xtrain, Xtest))
    yfinal = np.concatenate((ytrain, ytest))
    
    filename = 'svhn_format2/extra_32x32.mat'
    data = scipy.io.loadmat(filename)
    Xextra = data['X']
    Xextra = np.rollaxis(Xextra, 3)
    yextra = data['y']
    yextra[yextra == 10] = 0
    yextra = yextra.reshape((len(yextra), 1))
    print(Xextra.shape)
    print(yextra.shape)

    print('final')
    Xfinal2 = np.concatenate((Xfinal, Xextra))
    yfinal2 = np.concatenate((yfinal, yextra))
    print(Xfinal2.shape)
    print(yfinal2.shape)
    filename = 'digits.mat'
    data = {
        'X': Xfinal2,
        'y': yfinal2
    }
    data = scipy.io.savemat(filename, data)
createCombinedFormat2Matfile()

(73257, 32, 32, 3)
(73257, 1)
(26032, 32, 32, 3)
(26032, 1)
(531131, 32, 32, 3)
(531131, 1)
final
(630420, 32, 32, 3)
(630420, 1)


In [2]:
def putnondigitsintomat():
    filenames = glob.glob('alldata/*.png')
    images = []
    for i, filename in enumerate(filenames):
        img = cv2.imread(filename)
        images.append(img)
    
    filename = 'nondigits.mat'
    y = ['10' for i in range(len(images))]
    y = np.array(y)
    y = y.reshape((len(images), 1))
    data = {
        'X': images,
        'y': y
    }
    scipy.io.savemat(filename, data)

putnondigitsintomat()

In [4]:
def combineDigitsAndNonDigits():
    filename = 'digits.mat'
    data = scipy.io.loadmat(filename)
    Xdig = data['X']
    ydig = data['y']
    
    filename = 'nondigits.mat'
    data = scipy.io.loadmat(filename)
    Xnon = data['X']
    ynon = data['y']

    Xfinal = np.concatenate((Xdig, Xnon))
    yfinal = np.concatenate((ydig, ynon))
    print(Xfinal.shape)
    print(yfinal.shape)
    filename = 'combined.mat'
    data = {
        'X': Xfinal,
        'y': yfinal
    }
    data = scipy.io.savemat(filename, data)

combineDigitsAndNonDigits()


(866612, 32, 32, 3)
(866612, 1)


In [4]:
from sklearn.model_selection import train_test_split
filename = 'combined.mat'
data = scipy.io.loadmat(filename)
X = data['X']
y = data['y']

Xtrainval, Xtest, ytrainval, ytest = train_test_split(X, y, test_size=0.30, random_state=42)
print('Training/Validation set size: ', len(Xtrainval))
print('Testing set size: ', len(Xtest))

Xtrain, Xval, ytrain, yval = train_test_split(Xtrainval, ytrainval, test_size=0.30, random_state=42)
print('Training set size: ', len(Xtrain))
print('Validation set size: ', len(Xval))
    
    

Training/Validation set size:  606628
Testing set size:  259984
Training set size:  424639
Validation set size:  181989


In [9]:
def separateData(folder, X, y, count):
    y = y.flatten()
    for i, img in enumerate(X):
        label = int(y[i])
#         print('#{}#'.format(label))
        cv2.imwrite('{}/{}/{}.png'.format(folder, label, count), img)
        count += 1
    return count

count = 0
count = separateData('train', Xtrain, ytrain, count)
print("Separated train")
count = separateData('validation', Xval, yval, count)
print("Separated validation")
count = separateData('test', Xtest, ytest, count)


Separated train
Separated validation
